<a href="https://colab.research.google.com/github/swang243/handson-ml2/blob/master/Copy_of_04_training_linear_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

**Week 4 Assignment**

_This notebook is the starter code for exercise 4_

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Load Data

I loaded the data from an S3 bucket on AWS for you.

In [0]:
import pandas as pd
banks=pd.read_csv("https://s3.amazonaws.com/www.ruxton.ai/bank+defaults.csv")
banks.head()

,Financial Institution Name,total_assets,defaulter,Failure_Assistance,log_TA,NI_to_TA,Equity_to_TA,NPL_to_TL,REO_to_TA,ALLL_to_TL,core_deposits_to_TA,brokered_deposits_to_TA,liquid_assets_to_TA,loss_provision_to_TL,ROA,NIM,assets_growth,term_spread,stock_mkt_growth,real_gdp_growth,unemployment_rate_change,treasury_yield_3m,bbb_spread,bbb_spread_change,log_TA_plus_1Q,NI_to_TA_plus_1Q,Equity_to_TA_plus_1Q,NPL_to_TL_plus_1Q,REO_to_TA_plus_1Q,ALLL_to_TL_plus_1Q,core_deposits_to_TA_plus_1Q,brokered_deposits_to_TA_plus_1Q,liquid_assets_to_TA_plus_1Q,loss_provision_to_TL_plus_1Q,ROA_plus_1Q,NIM_plus_1Q,assets_growth_plus_1Q,max_total_assets
0,COLORADO NATIONAL BANK,142138,1,FAILURE,-0.048734,-6.505183,-0.759562,0.880756,-0.162301,-0.039358,-1.594099,-0.182975,0.095380,0.003640,-6.505183,-0.554017,0.365436,0.155788,-0.317435,-0.253927,0.859609,0.006692,0.894064,0.417366,-0.069667,-0.367905,-0.795966,2.013862,-0.066290,0.294916,-2.007894,-0.182975,0.106776,0.004823,-0.367905,-0.840312,-0.582232,142138
1,"COMMUNITY BANK, NATIONAL ASSOCIATION",6364903,0,NaN,2.754852,0.143005,0.249275,-0.405931,-0.327268,-0.244766,0.265125,-0.150770,0.229372,-0.003401,0.143005,-0.109970,1.447333,0.897368,-0.132755,0.348841,0.202649,-0.956664,-0.324746,-0.095051,2.767996,0.205816,0.236031,-0.388327,-0.335160,-0.245810,0.283318,-0.156056,0.318976,-0.003057,0.205816,-0.140365,0.003962,8494486
2,"NEXBANK, SSB",784781,0,NaN,1.211269,0.229021,-0.301031,-0.750449,-0.304600,-0.661804,-2.362597,-0.182975,-0.473225,-0.002684,0.229021,-0.783150,1.466905,-0.400396,0.188968,-0.387875,-0.782792,-0.896455,0.081524,0.161157,1.241315,0.470935,-0.242253,-0.809730,-0.374868,-0.673111,-1.386875,-0.182975,-0.630034,-0.003506,0.470935,-0.799310,0.284815,2698563
3,COMMERCE BANK/SHORE NATIONAL ASSOCIATION,2173481,0,NaN,1.962490,0.253469,-0.922810,-0.752942,-0.374868,-0.260532,1.438397,-0.182975,1.850473,-0.002771,0.253469,-0.099539,0.459243,0.433881,1.674053,0.951609,0.531129,-0.354567,-0.189323,-0.863677,2.043363,0.225454,-1.039013,-0.740467,-0.374868,-0.249210,1.427099,-0.182975,2.028526,-0.002819,0.225454,-0.324276,1.091618,2697759
4,CENTENNIAL BANK,241440,1,FAILURE,0.341981,-1.857315,-0.643992,4.583630,10.678175,0.926359,-2.175126,-0.182975,-1.471494,0.041009,-1.857315,-1.160576,1.145982,1.082763,1.685831,-1.928282,3.158971,-0.836245,3.331682,-3.169554,0.315538,-0.945865,-0.677217,4.701457,12.485022,-0.786735,-1.679305,-0.182975,-1.417561,-0.004326,-0.945865,-1.657427,-0.679579,241440


#EDA:

Feature Selection using RFE (Recursive Feature Elimination)

In [0]:
use=list(banks.columns.values[4:])  # use all the predictor data for example
X = banks[ use]  # petal width
X.columns
Y = banks["defaulter"]  # 1 if Iris virginica, else 0

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(banks[use], banks["defaulter"]):
    strat_train_set = banks.loc[train_index]
    strat_test_set = banks.loc[test_index]

X=strat_train_set[use]
y=strat_train_set["defaulter"]
X_test=strat_test_set[use]
Y_actual=strat_test_set["defaulter"]

In [0]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR

estimator = SVR(kernel="linear")
selector = RFE(estimator, 10, step=1)
selector = selector.fit(X, y)

print(selector.support_)
print(selector.ranking_)

# Logistic regression

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(banks[use], banks["defaulter"]):
    strat_train_set = banks.loc[train_index]
    strat_test_set = banks.loc[test_index]

X=strat_train_set[use]
y=strat_train_set["defaulter"]
X_test=strat_test_set[use]
Y_actual=strat_test_set["defaulter"]

In [0]:
import numpy as np
from scipy.stats import norm
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

In [0]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="lbfgs", random_state=42)

log_reg.fit(X, y)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(log_reg.predict(X_test),Y_actual))



print(logit_pvalue(log_reg.fit(X, y),X))

That looks awful. Let's try and dientify predictors that are intrinsic to banks balance sheet.

In [0]:
bs_cols = ['log_TA','NI_to_TA', 'Equity_to_TA', 'NPL_to_TL', 'REO_to_TA', 
              'ALLL_to_TL', 'core_deposits_to_TA', 'brokered_deposits_to_TA', 
              'liquid_assets_to_TA', 'loss_provision_to_TL', 
              'ROA', 
              'NIM', 'assets_growth']
X=strat_train_set[bs_cols]

X_test=strat_test_set[bs_cols]
log_reg.fit(X, y)
print(confusion_matrix(log_reg.predict(X_test),Y_actual))
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression



print(logit_pvalue(log_reg.fit(X, y),X))

That looks better. Now try some other methods like random forest SVM, xgboost, decisio trees. Try tuning them. Which do you choose?